# SmartFigure parameterized-plot tests

This notebook is a **hands-on test suite** for the parameter-aware `SmartFigure` design:

- SymPy expression → independent variable + parameter detection
- Deterministic parameter ordering for `numpify(args=...)`
- `SmartParameter` callback contract (`what_changed` tuple, clamping semantics)
- Per-figure slider UI policy (show only parameters used by that figure)
- Recompute only the plots affected by a parameter change

Run the notebook top-to-bottom in **JupyterLab**.


## Stages covered by this notebook

This notebook is organized to mirror the implementation stages:

1. `SmartParameter` dynamic properties + `what_changed` semantics
2. Expression analysis (constants, parameterized constants, multi-symbol rules)
3. Registry integration + deterministic ordering + evaluation contract
4. Figure-level recompute isolation (only dependent plots update)
5. Figure UI policy for sliders (only currently-used parameters)


## Imports and environment

This cell tries a couple common import layouts. Adjust the imports here if your project uses different module paths.


In [1]:
import gu_toolkit
gu_toolkit.setup()   # injects SymPy + plugin exports into notebook namespace

print("Environment:")
print("  Python:", sys.version.split()[0])
print("  SymPy:", sp.__version__)
print("  NumPy:", np.__version__)


🔧 Initializing GU Toolkit...
✓ Matplotlib backend set to 'widget'
[gu_toolkit] Skip (disabled) gu_toolkit.plugins.example
[gu_toolkit] Loaded gu_toolkit.plugins.NamedFunction (exports=1, hook=False)
[gu_toolkit] Loaded gu_toolkit.plugins.SmartException.SmartException (exports=1, hook=True)
[gu_toolkit] Loaded gu_toolkit.plugins.SmartFigure.SmartFigure (exports=9, hook=False)
[gu_toolkit] Loaded gu_toolkit.plugins.SmartParameters.SmartParameters (exports=4, hook=False)
[gu_toolkit] Loaded gu_toolkit.plugins.numpify (exports=1, hook=False)
✅ Smart Exception Handler Activated.
✅ Smart Exception Handler Activated.
[gu_toolkit] Exported 951 names into the notebook namespace.
🎓 GU Toolkit Ready.
Environment:
  Python: 3.13.9
  SymPy: 1.14.0
  NumPy: 2.1.3


## Stage 1 — `SmartParameter` contract

These tests check:

- `value` coercion + clamping
- `min`/`max` are dynamic and notify observers
- `what_changed` is a **tuple** listing all changes in a single operation
- changing bounds that clamp the value must include `'value'` in `what_changed`
- `step` stays unchanged when bounds change


In [2]:

# Basic construction
a = sp.Symbol("a")
p = SmartParameter(id=a, value=None, min_val=-1.0, max_val=1.0, default_val=0.0)

assert p.value == 0.0
print("✓ default value behavior OK")

# Coercion + clamping
p.value = "0.25"
assert isinstance(p.value, float) and abs(p.value - 0.25) < 1e-12

p.value = 10
assert p.value == 1.0

p.value = -10
assert p.value == -1.0

print("✓ coercion + clamping OK")

# Collect callback events
events = []
def cb(param, *, what_changed=None, owner_token=None, **kwargs):
    events.append(tuple(what_changed) if what_changed is not None else None)

tok = p.register_callback(cb)

# value change must report ("value",)
events.clear()
p.value = 0.5
assert events and events[-1] == ("value",), events
print("✓ what_changed for value OK")

# Bounds update: must notify and include value if clamped.
# This assumes the new API: p.set_bounds(min_val=..., max_val=...)
events.clear()
p.value = 0.9
p.set_bounds(min_val=-0.2, max_val=0.2)  # clamps value from 0.9 -> 0.2
assert events, "No callback fired on bounds update"
assert "min" in events[-1] and "max" in events[-1] and "value" in events[-1], events[-1]
print("✓ what_changed includes min/max/value when clamping OK")

# Step policy (assumes p.step exists)
p2 = SmartParameter(id=sp.Symbol("b"), min_val=-1.0, max_val=1.0, value=0.0)
initial_step = getattr(p2, "step", None)
assert initial_step is not None, "Expected SmartParameter.step to exist"
p2.set_bounds(min_val=-10.0, max_val=10.0)
assert abs(p2.step - initial_step) < 1e-15
print("✓ step unchanged on bounds update OK")


✓ default value behavior OK
✓ coercion + clamping OK
✓ what_changed for value OK
✓ what_changed includes min/max/value when clamping OK
✓ step unchanged on bounds update OK


## Stage 2 — Symbol selection and parameter detection

We test the rules:

- `|S|=0`: constant expression
- `|S|=1` and no symbol provided: that symbol is the independent variable
- `|S|=1` and symbol provided **different**: parameterized constant
- `|S|>1`: user must specify `symbol=...`


In [3]:

x = sp.Symbol("x")
a = sp.Symbol("a")
b = sp.Symbol("b")

fig = SmartFigure(var=x, show_now=False)

In [4]:
fig.widget

    'data': [],
    'layout': {'margin': {'b': 40, 'l': 40, 'r': 20, 't': 20},
 …

In [5]:


# |S| = 0 constant
p0 = fig.plot(sp.Integer(5),symbol=x, name="const5")
# data should be y=5 across x
x0, y0 = p0.compute_data(fig.x_range, 50)
assert len(x0) == len(y0) and len(y0) > 0
assert np.allclose(y0, 5.0)
print("✓ constant expression behaves as constant plot OK")


✓ constant expression behaves as constant plot OK


In [6]:

# |S|=1, no symbol: symbol is that free symbol
p1 = fig.plot(sp.sin(x), name="sinx")
assert p1.symbol == x
print("✓ |S|=1 and no symbol → that symbol is independent OK")

✓ |S|=1 and no symbol → that symbol is independent OK


In [7]:
# |S|=1, symbol specified different: parameterized constant in x, parameter a
p2 = fig.plot(a, name="param_const", symbol=x)
# Should compile and evaluate to constant y=a.value (default 0)
x2, y2 = p2.compute_data(fig.x_range, 10)
assert np.allclose(y2, 0.0), y2[:5]
print("✓ parameterized constant compiles and evaluates OK")


✓ parameterized constant compiles and evaluates OK


In [10]:

# |S|>1 requires explicit symbol
expr = sp.sin(x) * a + b
try:
    fig.plot(expr, name="bad_multi")  # no symbol
    raise AssertionError("Expected an error for |S|>1 without symbol")
except Exception as e:
    print("✓ multi-symbol without symbol raises:", type(e).__name__)


✓ multi-symbol without symbol raises: GuideError


In [11]:

p3 = fig.plot(expr, name="good_multi", symbol=x)
print("✓ multi-symbol with explicit symbol OK")


✓ multi-symbol with explicit symbol OK


## Stage 3 — Registry integration and deterministic ordering

This checks that parameters are created/reused in the registry and that the evaluation order is deterministic:

- the plot must remember `(symbol, *params_sorted_by_name)`
- `numpify(..., args=...)` must use that explicit ordering


In [17]:

# Create a fresh figure with an explicit registry (if supported)
reg = SmartParameterRegistry()
fig = SmartFigure(var=x, show_now=False, parameter_registry=reg)  # adjust if ctor differs
fig.widget



    'data': [],
    'layout': {'margin': {'b': 40, 'l': 40, 'r': 20, 't': 20},
 …

In [18]:
p = fig.plot(sin(x) * b + a, name="order_test", symbol=x)

# Registry should contain a and b
assert reg.get(a) is not None and reg.get(b) is not None
print("✓ registry auto-creates parameters OK")


✓ registry auto-creates parameters OK


In [19]:

# Deterministic param ordering by name: a then b
params = getattr(p, "param_symbols", None)
assert params is not None, "Expected Plot.param_symbols"
assert [s.name for s in params] == ["a", "b"], params
print("✓ deterministic parameter ordering OK")


✓ deterministic parameter ordering OK


In [32]:

# Changing only 'a' should shift plot vertically by the same constant.
reg.get(a).value = 0.0
xv, yv0 = p.compute_data(fig.x_range, 100)

reg.get(a).value = 2.0
xv, yv1 = p.compute_data(fig.x_range, 100)

assert np.allclose(yv1, yv0, atol=2.0)
print("✓ evaluation uses registry values with correct ordering OK")


✓ evaluation uses registry values with correct ordering OK


## Stage 4 — Recompute isolation (only dependent plots update)

Create two plots with disjoint parameter dependencies and verify that changing `a` updates only plot A, not plot B.

This test requires the figure to update its backend traces when parameters change.


In [ ]:

reg = SmartParameterRegistry()
fig = SmartFigure(var=x, show_now=False, parameter_registry=reg)

pA = fig.plot(sp.sin(a*x), name="depends_on_a", symbol=x)
pB = fig.plot(sp.cos(b*x), name="depends_on_b", symbol=x)

w = fig.widget  # force backend construction
backend = fig.backend

# Capture current y arrays from the backend (Plotly FigureWidget)
yA0 = np.array(backend.fig.data[0].y, dtype=float)
yB0 = np.array(backend.fig.data[1].y, dtype=float)

# Change a
reg.get(a).value = 0.1  # should trigger only pA recompute
yA1 = np.array(backend.fig.data[0].y, dtype=float)
yB1 = np.array(backend.fig.data[1].y, dtype=float)

assert not np.allclose(yA1, yA0), "Plot A did not change after changing a"
assert np.allclose(yB1, yB0), "Plot B changed even though it does not depend on a"
print("✓ recompute isolation OK (only dependent plots update)")


## Stage 5 — Figure slider UI policy

This checks the **per-figure** rule:

- show sliders only for parameters used by that figure’s current plots
- removing the last plot that uses a parameter removes that slider from the figure UI
- parameters are **not** removed from the registry


In [ ]:

reg = SmartParameterRegistry()
fig = SmartFigure(var=x, show_now=False, parameter_registry=reg)

# Add a plot that uses 'a'
p1 = fig.plot(a*sp.sin(x), name="uses_a", symbol=x)
w = fig.widget

# The widget should contain a slider area (exact structure may differ).
children = getattr(w, "children", ())
assert children, "Expected figure widget to have children"
print("Widget children types:", [type(c).__name__ for c in children])

# Heuristic: find slider-like objects (SmartSlider or ipywidgets container)
# Update this block if your implementation exposes fig._sliders or fig.sliders explicitly.
slider_count = 0
def _walk(node):
    global slider_count
    if node is None:
        return
    tname = type(node).__name__
    if tname in ("SmartSlider", "FloatSlider"):
        slider_count += 1
    for ch in getattr(node, "children", ()) or ():
        _walk(ch)

slider_count = 0
_walk(w)
assert slider_count >= 1, "Expected at least one slider in UI for parameter a"
print("✓ slider appears for used parameter")

# Remove the plot. Slider should disappear from UI, but registry keeps parameter.
fig.remove("uses_a")

# If widget is cached, implementation should rebuild slider area or update it in place.
w2 = fig.widget
slider_count = 0
_walk(w2)
assert slider_count == 0, f"Expected no sliders after removing plot; saw {slider_count}"
assert reg.get(a) is not None, "Registry should retain parameter a even if unused"
print("✓ slider removed when unused, registry retains parameter")
